In [0]:
CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.sales_bronze
AS SELECT * FROM STREAM read_files(
  "/Volumes/cgi_dev/naval/sales",
  format => "csv"
);

CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.products_bronze
AS SELECT * FROM STREAM read_files(
  "/Volumes/cgi_dev/naval/products",
  format => "csv"
);
CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.customers_bronze
AS SELECT * FROM STREAM read_files(
  "/Volumes/cgi_dev/naval/customers",
  format => "csv"
);

In [0]:
CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.sales_cleaned(
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
)
AS
SELECT distinct * from stream(cgi_dev.navaneeth_bronze.sales_bronze);

CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.products_cleaned
AS
SELECT distinct * from stream(cgi_dev.navaneeth_bronze.products_bronze);

CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_bronze.customers_cleaned
AS
SELECT distinct * from stream(cgi_dev.navaneeth_bronze.customers_bronze);

In [0]:
CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_silver.product_silver;

APPLY CHANGES INTO
  cgi_dev.navaneeth_silver.product_silver
FROM
  stream(cgi_dev.navaneeth_bronze.products_cleaned)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum,_rescued_data)
STORED AS
  SCD TYPE 1;

In [0]:

CREATE OR REFRESH STREAMING TABLE cgi_dev.navaneeth_silver.customers_silver;

APPLY CHANGES INTO
  cgi_dev.navaneeth_silver.customers_silver
FROM
  stream(cgi_dev.navaneeth_bronze.customers_cleaned)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum,_rescued_data)
STORED AS
  SCD TYPE 2;


In [0]:
create or refresh materialized view cgi_dev.navaneeth_gold.customers_active as
select * from cgi_dev.navaneeth_silver.customers_silver where __END_AT is null

In [0]:

create or replace materialized view cgi_dev.navaneeth_gold.sales_details as 
(SELECT 
    o.order_id,
    o.order_date,
    o.customer_id,
    c.customer_name,
    c.customer_email,
    c.customer_city,
    o.product_id,
    p.product_name,
    p.product_category,
    p.product_price,
    o.quantity,
    o.discount_amount,
    o.total_amount
FROM cgi_dev.navaneeth_bronze.sales_cleaned o
JOIN cgi_dev.navaneeth_silver.product_silver p
  ON o.product_id = p.product_id
JOIN (
      SELECT DISTINCT customer_id, customer_name, customer_email, customer_city 
      FROM cgi_dev.navaneeth_gold.customers_active
     ) c
  ON o.customer_id = c.customer_id)